In [1]:
import pandas as pd
import numpy as np

#### Hi guys, I'm writing this on the plane without internet but hope this will give more insight if things are still unclear.

Im using pandas to illustrate the ideas but am so so sure this is not perfect.

These are the general steps:
1. Pass over unordered data and determine and track what lithology is actually there (using the additional ref# column)
2. Slice the data structure on this new information to make or 'blocks' of each lithology.
3. Build the ordered set (its not neccessary to make a seperate data structure, im sure you can rearange inplace for more space efficiency but its easier to demonstrate with pandas so don't judge me)
4. Make a daddy object to keep track of start and ends of blocks in the ordered set.

In [2]:
# Let's first set up some dummy data
data = [
    ['sandstone', 792751, 480715, 'Sb', -1],
    ['basalt', 792751, 480715, 'Au', -1],
    ['bobs_rock', 792751, 480715, 'Au', -1],
    ['mafic_basalt', 792751, 480715, 'Au', -1],
    ['bobsRockButNamedDumb', 792751, 480715, 'Au', -1],
    ['arenite', 792751, 480715, 'Au', -1],
]

# And some column names we will reuse
# lithology | lat | long | primary_commodity | litho_reference_Number
cols = "Lith Lat Long Prim_Comm ref#".split()

# And make a dataframe (kinda like a database table)
df = pd.DataFrame(columns=cols, data=data)
df

,Lith,Lat,Long,Prim_Comm,ref#
0,sandstone,792751,480715,Sb,-1
1,basalt,792751,480715,Au,-1
2,bobs_rock,792751,480715,Au,-1
3,mafic_basalt,792751,480715,Au,-1
4,bobsRockButNamedDumb,792751,480715,Au,-1
5,arenite,792751,480715,Au,-1


In [3]:
# Need to make the reference dictionary
dictionary = {"basalt" : ['basalt', 'mafic_basalt'],
      "sandstone" : ['sandstone', 'arenite'],
      "bob": ['bobs_rock', 'bobsRockButNamedDumb']}

# And keep the keys (whatever you decide to call some lithology) to iterate over
lithKeyNames = list(dictionary.keys())

print(dictionary)
print()
print(lithKeyNames)

{'basalt': ['basalt', 'mafic_basalt'], 'sandstone': ['sandstone', 'arenite'], 'bob': ['bobs_rock', 'bobsRockButNamedDumb']}

['basalt', 'sandstone', 'bob']


Then we pass over the unordered set and see what lithology each row refers to.

Lets set up a function to deal with properly assigning this.
This will inevitably be kinda costly because we know and assume nothing prior about the row.
Since Kavita is using python, I would recommend using the cython libs
that compile the clunkier shit into fast c.

Important though: Double for loops in python = really really bad.
Maybe setting up the dictionary itself with common liths at the start?

In [4]:
def confirmLithology(row):
    # for each of these ['basalt', 'sandstone', 'bob']
    for i in range(len(lithKeyNames)):
        actualLith = str(lithKeyNames[i])
        # check all the names associated for a match
        possibleNames = dictionary[actualLith]
        for name in possibleNames:
            # once one is found, set the relevant row ref# in the original dataframe
            # to the index of the actual lithology name
            if row['Lith'] == name:
                df['ref#'].iloc[row.name] = int(i)
                return

In [5]:
# Apply the above function to each row
df.apply(confirmLithology, axis=1)

C:\Users\user1\Miniconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0    None
1    None
2    None
3    None
4    None
5    None
dtype: object

In [6]:
# Create an empty frame (not ideal) with the same column headers
organisedData = pd.DataFrame(columns=cols)

# Then append 'blocks' sequentially to the new frame
for i in range(len(lithKeyNames)):
    subset = df[df['ref#'] == i]
    organisedData = organisedData.append(subset)

# Reset the index
organisedData = organisedData.reindex()
organisedData

,Lith,Lat,Long,Prim_Comm,ref#
1,basalt,792751,480715,Au,0
3,mafic_basalt,792751,480715,Au,0
0,sandstone,792751,480715,Sb,1
5,arenite,792751,480715,Au,1
2,bobs_rock,792751,480715,Au,2
4,bobsRockButNamedDumb,792751,480715,Au,2


And thats the idea behind building the data structure.
So now you just need some Lithology object class type thing that stores
where your 'block' is within the whole ordered set.

We could keep using the lithKeyNames list as above and refer to the index
of what is being queried as the ref# BUT a) as that list gets bigger
itll be slow ~O(n) and b) its is harder to break an object oriented approach,
especially if you guys take this further and bigger.

In [7]:
# Obviously we'd have to think about a better daddy class for a pageTable
# (and program structure in general tbh) than a python dict but just to illustrate:
keys = lithKeyNames
vals = range(len(lithKeyNames))
pageTable = dict(zip(keys, vals))
print(pageTable)

{'basalt': 0, 'sandstone': 1, 'bob': 2}


In [8]:
# Then query any of the enforced liths (by a button to avoid input errors)
# And oganise subsequent queries like so:
query = "sandstone"
returnedRows = organisedData[pageTable[query] == organisedData['ref#']]
returnedRows

,Lith,Lat,Long,Prim_Comm,ref#
0,sandstone,792751,480715,Sb,1
5,arenite,792751,480715,Au,1


In [9]:
primaryComm = 'Sb'
returnedRows[returnedRows['Prim_Comm'] == primaryComm]

,Lith,Lat,Long,Prim_Comm,ref#
0,sandstone,792751,480715,Sb,1
